In [5]:
import torch
from torchdiffeq import odeint
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import time, os, sys
import numpy as np
import pickle
import pandas as pd
from scripts.utils import set_seed
from scripts.ds_class import *
from scripts.homeos import *
from scripts.plotting import *
from scripts.fit_motif import *
from scripts.time_series import *
from scripts.ra import *

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator
import seaborn as sns
sns.set(style="darkgrid", palette="muted", font="serif")
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams['xtick.labelsize'] = 14  # font size
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams["font.family"] = "serif"
mpl.rcParams['pdf.fonttype'] = 42  # Use TrueType fonts (editable in Illustrator)
mpl.rcParams['ps.fonttype'] = 42   # Same for EPS
mpl.rcParams['svg.fonttype'] = 'none'  # Keep text as text in SVG

set_seed(313)
save_dir = 'experiments/nd_ra'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [6]:
def nd_ra_exp(random_seed=313, dims = [3,4,5,6,7,8,9,10], save_dir='experiments/nd_ra'):
    set_seed(random_seed)
    homeo_type = 'node'
    layer_sizes = 1*[128]
    homeo_mean = 0.01
    homeo_std = 0.25
    dt=.2
    time_span = torch.tensor([0.0, 2.]) 
    train_ratio = 0.8
    ds_motif = 'ring' 
    analytic = True
    vf_on_ring_enabled = True #if analytic then not used
    training_pairs = False
    alpha_init = None
    if training_pairs:
        time_span = torch.tensor([0.0, dt])
    simulation_params = {'initial_conditions_mode': 'random', 'number_of_target_trajectories': 50, 'time_span': time_span, 'dt': dt, 'noise_std': .0,
                        'training_pairs': training_pairs, 'margin': 0.1, 'seed': 42, 'train_ratio': train_ratio}
    
    homeo_type = 'node'
    layer_sizes = 1*[128]
    init_type = None

    lr = 0.01
    num_epochs = 1000
    annealing_params = {'dynamic': False, 'initial_std': .0, 'final_std': 0.}
    training_params = {'lr': lr, 'num_epochs': num_epochs, 'annealing_params': annealing_params, 'early_stopping_patience': 1000, "batch_size": 32,
                    'use_inverse_formulation': True}
    
    # all_parameters = {'target_homeo_params': target_homeo_params, 'homeo_params': homeo_params, 'training_params': training_params, 'simulation_params': simulation_params}
    # with open("parameters.pkl", "wb") as f:
    #     pickle.dump(all_parameters, f)
    
    results = []
    for dim_i, dim in enumerate(dims):
        print(f"Dimension: {dim}")

        # generate ring attractor trajectories
        generator_ra = AnalyticalRingAttractor(dim=dim, dt=dt, time_span=time_span) 
        init_conds = prepare_initial_conditions(dim=dim, mode=simulation_params['initial_conditions_mode'], num_points=simulation_params['number_of_target_trajectories'],
                                                 margin=simulation_params['margin'], seed=simulation_params['seed'])
        ra_trajs = generator_ra.compute_trajectory(torch.tensor(init_conds, dtype=torch.float32))

        B = ra_trajs.shape[0]
        n_train = int(train_ratio * B)
        n_test = B - n_train
        target_homeo_params = {'homeo_type': homeo_type, 'dim': dim, 'layer_sizes': layer_sizes, 'init_type': 'small', 'activation': nn.ReLU, 'init_mean': homeo_mean, 'init_std': homeo_std}
        target_homeo = build_homeomorphism(target_homeo_params)
        save_homeo_ds_net(target_homeo, f"{save_dir}/target_homeo_{dim}.pth")

        homeo_params = {'homeo_type': homeo_type, 'dim': dim, 'layer_sizes': layer_sizes, 'activation': nn.ReLU, 'init_type': init_type}
        ds_params = {'ds_motif': ds_motif, 'dim':dim, 'dt': dt, 'time_span': time_span, 'analytic': analytic, 'vf_on_ring_enabled': vf_on_ring_enabled, 'alpha_init': alpha_init}

        target_jacobian_norm = jacobian_norm_over_batch(target_homeo, ra_trajs.reshape(-1,dim))
        #print(f"Target Jacobian norm: {target_jacobian_norm}")

        trajectories_target_full = target_homeo(ra_trajs)
        trajectories_target_full, trajectories_target, mean, std = normalize_scale_pair(trajectories_target_full, simulation_params['training_pairs'])
        np.save(f"trajectories_target_{dim}.npy", trajectories_target_full.detach().numpy())

        train_set, test_set = random_split(trajectories_target_full, [n_train, n_test])
        trajectories_target_train = trajectories_target[train_set.indices]
        trajectories_target_test = trajectories_target[test_set.indices]

#        trajectories_target_train = trajectories_target_train.detach().numpy()
        #print(trajectories_target_train.shape)
        # plot_trajectories_3d([trajectories_target_train, trajectories_target_test.detach().numpy()], elev=45, azim=90)
        # fig, ax = plt.subplots(figsize=(4, 6))
        # ax.plot(trajectories_target_train[:, :, 0], trajectories_target_train[:, :, 1], 'o', markersize=2, color='blue', alpha=0.5, label='')
        # plt.show()
        # fig, ax = plt.subplots(figsize=(4, 6))
        # ax.plot(trajectories_target_train[:, :, 1], trajectories_target_train[:, :, 2], 'o', markersize=2, color='blue', alpha=0.5, label='')
        # for traj in trajectories_target_train:
        #     ax.plot(np.arange(traj.shape[0]), traj[:, 2], 'o', markersize=2, color='red', alpha=0.5)
        # plt.show()

        target_ra_points = get_homeo_invman(target_homeo, dim=dim)
        target_ra_points = (target_ra_points - mean.detach().numpy()) / std.detach().numpy() 

        homeo = build_homeomorphism(homeo_params)
        source_system_ra = build_ds_motif(**ds_params)
        homeo_ds_net = Homeo_DS_Net(homeo, source_system_ra)
        homeo_ds_net, losses, grad_norms = train_homeo_ds_net_batched(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target_train, **training_params)
        homeo_ds_net.eval()
        
        _, _, training_loss = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target_train)
        _, _, test_loss = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target_test)
        traj_src_np, traj_trans_np, _ = test_single_homeo_ds_net(homeo_ds_net=homeo_ds_net, trajectories_target=trajectories_target)

        
        fit_ra_points = get_homeo_invman(homeo_ds_net.homeo_network, dim=dim) 
        jac_norm = jacobian_norm_over_batch(homeo_ds_net.homeo_network, trajectories_target.reshape(-1,dim))

        results.append({
            "dim": dim,
            "train_loss": training_loss,
            "test_loss": test_loss,
            "jacobian_norm": jac_norm,
            "target_jacobian_norm": target_jacobian_norm,
            "losses": losses,
            "grad_norms": grad_norms,
            "fit_ra_points": fit_ra_points,
            "target_ra_points": target_ra_points[0],
        })

        save_homeo_ds_net(homeo_ds_net, f"{save_dir}/homeo_{dim}.pth")
        np.save(f"{save_dir}/traj_motif_transformed_{dim}.npy", traj_trans_np) #save trajectories_motif 

    df = pd.DataFrame(results)
    df.to_pickle(f"{save_dir}/summary_df.pkl")

In [ ]:
nd_ra_exp(dims=[3,4,5,6,7,8,9,10])

Dimension: 3
Model saved to experiments/nd_ra/target_homeo_3.pth
Computing Jacobian norms...
